# Text Classification - Lab

## Introduction

In this lab, we'll use everything we've learned so far to build a model that can classify a text document as one of many possible classes!

## Objectives

You will be able to:

- Perform classification using a text dataset, using sensible preprocessing, tokenization, and feature engineering scheme 
- Use scikit-learn text vectorizers to fit and transform text data into a format to be used in a ML model 



# Getting Started

For this lab, we'll be working with the classic **_Newsgroups Dataset_**, which is available as a training data set in `sklearn.datasets`. This dataset contains many different articles that fall into 1 of 20 possible classes. Our goal will be to build a classifier that can accurately predict the class of an article based on the features we create from the article itself!

Let's get started. Run the cell below to import everything we'll need for this lab. 

In [1]:
import nltk
from nltk.corpus import stopwords
import string
from nltk import word_tokenize, FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import numpy as np
np.random.seed(0)

Now, we need to fetch our dataset. Run the cell below to download all the newsgroups articles and their corresponding labels. If this is the first time working with this dataset, scikit-learn will need to download all of the articles from an external repository -- the cell below may take a little while to run. 

The actual dataset is quite large. To save us from extremely long runtimes, we'll work with only a subset of the classes. Here is a list of all the possible classes:

<img src='classes.png'>

For this lab, we'll only work with the following five:

* `'alt.atheism'`
* `'comp.windows.x'`
* `'rec.sport.hockey'`
* `'sci.crypt'`
* `'talk.politics.guns'`

In the cell below:

* Create a list called `categories` that contains the five newsgroups classes listed above, as strings 
* Get the training set by calling `fetch_20newsgroups()` and passing in the following parameters:
    * `subset='train'`
    * `categories=categories`
    * `remove=('headers', 'footers', 'quotes')` -- this is so that the model can't overfit to metadata included in the articles that sometimes acts as a dead-giveaway as to what class the article belongs to  
* Get the testing set as well by passing in the same parameters, with the exception of `subset='test` 

In [2]:
categories = ['alt.atheism','comp.windows.x','rec.sport.hockey','sci.crypt','talk.politics.guns']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories,remove=('headers','footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(categories=categories,remove=('headers','footers', 'quotes'))

Great! Let's break apart the data and the labels, and then inspect the class names to see what the actual newsgroups are.

In the cell below:

* Grab the data from `newsgroups_train.data` and store it in the appropriate variable  
* Grab the labels from `newsgroups_train.target` and store it in the appropriate variable  
* Grab the label names from `newsgroups_train.target_names` and store it in the appropriate variable  
* Display the `label_names` so that we can see the different classes of articles that we're working with, and confirm that we grabbed the right ones  

In [3]:
data = newsgroups_train.data
target = newsgroups_test.data
label_names = newsgroups_train.target_names
label_names

['alt.atheism',
 'comp.windows.x',
 'rec.sport.hockey',
 'sci.crypt',
 'talk.politics.guns']

Finally, let's check the shape of `data` to see what our data looks like. We can do this by checking the `.shape` attribute of `newsgroups_train.filenames`.

Do this now in the cell below.

In [5]:
# Your code here
newsgroups_train.filenames.shape

(2814,)

Our dataset contains 2,814 different articles spread across the five classes we chose. 

### Cleaning and Preprocessing Our Data

Now that we have our data, the fun part begins. We'll need to begin by preprocessing and cleaning our text data. As you've seen throughout this section, preprocessing text data is a bit more challenging that working with more traditional data types because there's no clear-cut answer for exactly what sort of preprocessing and cleaning we need to do. Before we can begin cleaning and preprocessing our text data, we need to make some decisions about things such as:

* Do we remove stop words or not?
* Do we stem or lemmatize our text data, or leave the words as is?
* Is basic tokenization enough, or do we need to support special edge cases through the use of regex?
* Do we use the entire vocabulary, or just limit the model to a subset of the most frequently used words? If so, how many?
* Do we engineer other features, such as bigrams, or POS tags, or Mutual Information Scores?
* What sort of vectorization should we use in our model? Boolean Vectorization? Count Vectorization? TF-IDF? More advanced vectorization strategies such as Word2Vec?


These are all questions that we'll need to think about pretty much anytime we begin working with text data. 

Let's get right into it. We'll start by getting a list of all of the english stopwords, and concatenating them with a list of all the punctuation. 

In the cell below:

* Get all the english stopwords from `nltk` 
* Get all of the punctuation from `string.punctuation`, and convert it to a list 
* Add the two lists together. Name the result `stopwords_list` 
* Create another list containing various types of empty strings and ellipses, such as `["''", '""', '...', '``']`. Add this to our `stopwords_list`, so that we won't have tokens that are only empty quotes and such  

In [7]:
nltk.download('stopwords')
stopwords_list = stopwords.words('english') + list(string.punctuation)
stopwords_list += ["''", '""', '...', '``']

[nltk_data] Downloading package stopwords to /home/enthusiastic-
[nltk_data]     constructor-3429/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Great! We'll leave these alone for now, until we're ready to remove stop words after the tokenization step. 

Next, let's try tokenizing our dataset. In order to save ourselves some time, we'll write a function to clean our dataset, and then use Python's built-in `map()` function to clean every article in the dataset at the same time. 

In the cell below, complete the `process_article()` function. This function should:

* Take in one parameter, `article` 
* Tokenize the article using the appropriate function from `nltk` 
* Lowercase every token, remove any stopwords found in `stopwords_list` from the tokenized article, and return the results 

In [11]:
def process_article(article):
    token = nltk.word_tokenize(article)
    stop_words_removed = [i.lower() for i in token if i.lower() not in stopwords_list]
    return stop_words_removed

Now that we have this function, let's go ahead and preprocess our data, and then move into exploring our dataset. 

In the cell below:

* Use Python's `map()` function and pass in two parameters: the `process_article` function and the `data`. Make sure to wrap the whole map statement in a `list()`.

**_Note:_** Running this cell may take a minute or two!

In [12]:
nltk.download('punkt')
processed_data = list(map(process_article, data))

[nltk_data] Downloading package punkt to /home/enthusiastic-
[nltk_data]     constructor-3429/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Great. Now, let's inspect the first article in `processed_data` to see how it looks. 

Do this now in the cell below.

In [13]:
processed_data[0]

['note',
 'trial',
 'updates',
 'summarized',
 'reports',
 '_idaho',
 'statesman_',
 'local',
 'nbc',
 'affiliate',
 'television',
 'station',
 'ktvb',
 'channel',
 '7',
 'randy',
 'weaver/kevin',
 'harris',
 'trial',
 'update',
 'day',
 '4',
 'friday',
 'april',
 '16',
 '1993',
 'fourth',
 'day',
 'trial',
 'synopsis',
 'defense',
 'attorney',
 'gerry',
 'spence',
 'cross-examined',
 'agent',
 'cooper',
 'repeated',
 'objections',
 'prosecutor',
 'ronald',
 'howen',
 'spence',
 'moved',
 'mistrial',
 'denied',
 'day',
 'marked',
 'caustic',
 'cross-examination',
 'deputy',
 'marshal',
 'larry',
 'cooper',
 'defense',
 'attorney',
 'gerry',
 'spence',
 'although',
 'spence',
 'explicitly',
 'stated',
 'one',
 'angle',
 'stategy',
 'must',
 'involve',
 'destroying',
 'credibility',
 'agent',
 'cooper',
 'cooper',
 'government',
 "'s",
 'eyewitness',
 'death',
 'agent',
 'degan',
 'spence',
 'attacked',
 'cooper',
 "'s",
 'credibility',
 'pointing',
 'discrepancies',
 'cooper',
 "'s",
 '

Now, let's move onto exploring the dataset a bit more. Let's start by getting the total vocabulary size of the training dataset. We can do this by creating a `set` object and then using it's `.update()` method to iteratively add each article. Since it's a set, it will only contain unique words, with no duplicates. 

In the cell below:

* Create a `set()` object called `total_vocab` 
* Iterate through each tokenized article in `processed_data` and add it to the set using the set's `.update()` method 
* Once all articles have been added, get the total number of unique words in our training set by taking the length of the set 

In [ ]:
processed_data[:5]

total_vocab = set()
for note in processed_data:
    for word in note:
        print(word)

note
trial
updates
summarized
reports
_idaho
statesman_
local
nbc
affiliate
television
station
ktvb
channel
7
randy
weaver/kevin
harris
trial
update
day
4
friday
april
16
1993
fourth
day
trial
synopsis
defense
attorney
gerry
spence
cross-examined
agent
cooper
repeated
objections
prosecutor
ronald
howen
spence
moved
mistrial
denied
day
marked
caustic
cross-examination
deputy
marshal
larry
cooper
defense
attorney
gerry
spence
although
spence
explicitly
stated
one
angle
stategy
must
involve
destroying
credibility
agent
cooper
cooper
government
's
eyewitness
death
agent
degan
spence
attacked
cooper
's
credibility
pointing
discrepancies
cooper
's
statements
last
september
made
court
cooper
conceded
things
compressed
seconds
's
difficult
remember
went
first
cooper
acknowledged
carried
9mm
colt
commando
submachine
gun
silenced
barrel
thought
colt
commando
revolver
cooper
continued
stating
federal
agents
specific
plans
use
weapon
started
kill
weaver
's
dog
spence
asked
seven
cartridges
could
f

--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
_/_/
_/
_/_/
_/_/_/_/
_/_/_/_/
_/_/
_/_/_/
_/
_/
_/
_/
_/
_/
_/
_/
_/
_/
_/
_/
_/
_/
_/
_/
_/
_/
_/
_/
_/_/_/
_/
_/_/_/_/
_/
_/_/_/
_/_/_/
_/_/
_/
_/
_/
_/
_/
_/
_/
_/
_/
_/
_/
_/
_/
_/
_/
_/
_/
_/
_/
_/
_____
_/_/_/
_/_/_/_/
_/
_/
_/_/_/_/
_/_/_/_/
_/
_/
_/_/_/
/____/
marc
foster
r.s.h
contact
oklahoma
city
blazers
1993
central
hockey
university
oklahoma
geography
department
league
adams
cup
internet
mfoster
geohub.gcn.uoknor.edu
champions
mfoster
alliant.backbone.uoknor.edu
placed
chl
mailing
list
send
email
either
address
export.lcs.mit.edu
/pub/sunkbd..930314.tar.z
/maf
march/april
version
x
journal
open
front
'll
working
programming
x-clients
summer
since
n't
much
experience
programming
x
thought
issue
might
helpful
section
debugging
section
40
common
errors
programming
x
end
errors
section
following
references
tutorials
x
programming
style
rosenthal
david
simple
x11
client
program
proceedings
winter
1988
usenix
conference
1988
lemke
d.

started
performing
better
offensively
converted
center
wing
although
lefty
sanderson
went
left
wing
righty
yake
went
right
side
biggest
disappointment
hands
john
cullen
cullen
disasterous
77
point
season
last
year
first
full
season
trade
cullen
started
season
summer
back
surgery
fell
flat
face
appropriate
since
spent
whaler
career
flat
ass
whining
cullen
scored
9
point
19
games
clubhouse
malcontent
commanded
powerplay
9
success
percentage
21
sanderson
sulked
way
town
worst
4
year
4m
contract
three
years
left
run
one
would
give
2nd
round
draft
pick
maple
leafs
offered
hartford
honorable
mention
steve
konroyd
also
subpar
signing
3
year
2.1m
contract
eric
weinrich
showed
flashes
competence
overall
played
poorly
jim
mckenzie
much
better
hockey
player
two
seasons
ago
frank
pietrangelo
seemed
play
well
sean
burke
extended
period
got
make
number
starts
row
according
osf/motif
style
guide
one
use
cursor
shapes
give
user
visual
clue
happening
expected
hourglass
cursor
shown
application
busy
cau

pts
pt
--
--
--
--
--
--
--
--
--
--
feb.
41
61
3-
8-
1
7
pts
year
246
268
march
02
la
6
vs
calgary
2
16,005
28-29-
7
63
pts
pt
04
la
8
vs
ottawa
6
16,005
29-29-
7
65
pts
pt
06
la
6
vs
edmonton
1
16,005
30-29-
7
67
pts
pt
09
la
3
ny
rangers
4
18,200
30-30-
7
67
pts
pt
11
la
3
pittsburgh
4
16,164
30-31-
7
67
pts
pt
13
philadelphia
postponed
1
period
1-1
due
weather
resch
4/1
14
buffalo
postponed
due
weather
rescheduled
3/15
15
la
4
buffalo
2
13,799
31-31-
7
69
pts
pt
16
la
8
vs
winnipeg
4
16,005
32-31-
7
71
pts
pt
18
la
7
vs
ny
islanders
4
16,005
33-31-
7
73
pts
pt
20
la
3
vs
st
louis
2
16,005
34-31-
7
75
pts
pt
24
la
2
vancouver
6
16,150
34-32-
7
75
pts
5
26
la
4
edmonton
1
17,503
35-32-
7
77
pts
5
28
la
3
winnipeg
3
15,566
35-32-
8
78
pts
pt
29
la
9
detroit
3
19,875
36-32-
8
80
pts
5
31
la
5
toronto
5
15,720
36-32-
9
81
pts
pt
--
--
--
--
--
--
--
--
--
--
mar
71
47
9-
3-
2
20
pts
year
317
315
april
01
la
3
philadelphia
1
17,380
37-32-
9
83
pts
pt
rescheduled
03
la
0
vs
minnesota
3
16

7/90
two
versions
xprompt
posted
comp.sources.x
latter
unauthorized
rewrite
r.
forsman
thoth
reef.cis.ufl.edu
1/91
version
xmenu
available
comp.sources.x
worked
likely
re-released
xp-1.1.tar.z
xpick-1.0.tar.z
xzap-1.0.tar.z
export
's
contrib/
tools
gerry.tomlinson
newcastle.ac.uk
act
x
versions
simple
display
choice-making
tools
k
p
4/92
xtpanel
lets
user
build
panel
containing
interactive
objects
buttons
sliders
text
fields
etc.
either
command
line
using
simple
scripting
language
available
anonymous
ftp
hanauma.stanford.edu
36.51.0.16
pub/x/xtpanel.tar.z
may
also
found
alt.sources
archives
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
subject
90
get
x-based
debugger
xdbx
x
interface
dbx
debugger
available
via
ftp
export
current
1/91
version
2.1
patchlevel
2
x
interface
gdb
called
xxgdb
like
xdbx
2.1.2
part
comp.sources.x
volume
11
2/91
xxgdb-1.06.tar.z
export
mxgdb
motif
interface
gdb
jim
tsillas
jtsillas
bubba.ma30.bull.com
v

...

prohibits
drinking
person
says
muslim
proceeds
drink
bottle
beer
mean
islam
teaches
people
drink
beer
course
idiot
would
think
see
point
judging
islam
capitalist
terms
capitalism
ideology
based
largely
assumption
people
want
maximise
wealth
--
assumption
opposition
islamic
teachings
say
islam
bad
capitalist
pretty
unthinking
--
islam
pretend
capitalist
try
capitalist
mean
islam
support
free-market
--
general
--
parts
capitalism
opposed
islam
understand
one
postulate
numerous
reasons
theory
islam
secularist
capitalist
etc
etc
selim
give
clear
historical
example
show
fallacy
views
think
obviously
islam
lack
education
power
large
part
history
islamic
world
powerful
significant
section
history
islamic
world
foremost
sciences
say
islam
example
anti-education
completely
absurd
try
blame
situation
islam
--
history
shows
conclusion
false
instead
must
reasons
situation
well
selim
viewpoint
women
islam
makes
question
extent
knowledge
islam
really
think
knowledgeable
enough
able
judge
whether
mus

greater
authentication
posted
information
hand
advances
ensuring
anonymity
remailers
forthcoming
see
privacy
_____
2.1
privacy
internet
generally
privacy
multiple
connotations
society
perhaps
even
internet
cyberspace
take
mean
exclusive
use
access
account
data
stored
directed
email
encounter
arbitrary
restrictions
searches
words
others
may
obtain
data
associated
account
without
permission
ideas
probably
fairly
limiting
liberal
scope
internet
users
consider
private
domains
users
n't
expect
want
privacy
expect
demand
_____
2.2
privacy
un
important
internet
somewhat
debatable
inflammatory
topic
arousing
passionate
opinions
internet
take
privacy
granted
rudely
surprised
find
tenuous
nonexistent
governments
rules
protect
privacy
illegal
search
seizure
clause
u.s.
constitution
adopted
others
many
antithetical
laws
prohibiting
secret
communications
allowing
wiretapping
rules
generally
carry
internet
specific
rules
governing
however
legal
repercussions
global
internet
still
largely
unknown
unt

safeguards
internet
growing
become
completely
global
international
superhighway
data
traffic
inevitably
entail
data
voice
messages
postal
mail
many
items
extremely
personal
nature
computer
items
many
people
consider
completely
private
local
hard
drives
literally
inches
global
network
connections
also
sensitive
industrial
business
information
exchanged
networks
currently
volume
may
conceivably
merge
internet
would
agree
basic
sensitive
uses
internet
significant
mechanisms
currently
place
ensure
much
privacy
new
standards
calling
uniform
introduction
privacy
enhanced
mail
pem
uses
encryption
technologies
ensure
privacy
privacy
protection
automatic
may
significantly
improve
safeguards
technology
extremely
destructive
privacy
surreptitious
surveilance
overwhelmingly
effective
protecting
e.g
encryption
government
agencies
opposed
unlimited
privacy
general
believe
lawfully
forfeited
cases
criminal
conduct
e.g
court-authorized
wiretapping
however
powerful
new
technologies
protect
privacy
comp

said
's
arrived
asked
whether
bobby
's
real
betcha
welcome
alt.atheism
rest
assured
gets
worse
pearls
wisdom
bobby
reproduce
anyone
keith
keeping
big
file
stuff
allah
's
infinite
wisdom
universe
created
nothing
saying
became
therefore
allah
exists
--
bobby
mozumder
proving
existence
allah
1
wait
said
humans
rarely
reasonable
n't
contradict
atheism
everything
explained
logic
reason
contradiction
atheism
proves
false
--
bobby
mozumder
proving
existence
allah
2
plus
believer
would
contradictory
quran
allah
exist
--
bobby
mozumder
proving
existence
allah
3
one
thing
relates
among
navy
men
get
tatoos
say
mom
love
mom
makes
virile
men
compare
homos
raised
study
get
point
--
bobby
mozumder
islamically
rigorous
alt.atheism
mmmmm
quality
*and*
quantity
new
voice
islam
pbuh
cheers
simon
know
least
one
person
list
says
first
heard
clipper
friday
morning
newspaper
another
already
fired
letter
protest
nist
point
suspect
list
interesting
various
reasons
represent
cabal
put
proposal
together
yes
othe

atheists
strong
atheists
atheists
believe
non-existence
gods
others
limit
atheism
specific
gods
christian
god
rather
making
flat-out
denials
n't
disbelieving
god
thing
believing
n't
exist
definitely
disbelief
proposition
means
one
believe
true
believing
something
true
equivalent
believing
false
one
may
simply
idea
whether
true
brings
us
agnosticism
agnosticism
term
'agnosticism
coined
professor
huxley
meeting
metaphysical
society
1876
defined
agnostic
someone
disclaimed
strong
atheism
believed
ultimate
origin
things
must
cause
unknown
unknowable
thus
agnostic
someone
believes
know
sure
whether
god
exists
words
slippery
things
language
inexact
beware
assuming
work
someone
's
philosophical
point
view
simply
fact
calls
atheist
agnostic
example
many
people
use
agnosticism
mean
weak
atheism
use
word
atheism
referring
strong
atheism
beware
also
word
atheist
many
shades
meaning
difficult
generalize
atheists
say
sure
atheists
n't
believe
god
example
certainly
case
atheists
believe
science
best

### Exploring Data With Frequency Distributions

Great -- our processed dataset contains 46,990 unique words! 

Next, let's create a frequency distribution to see which words are used the most! 

In order to do this, we'll need to concatenate every article into a single list, and then pass this list to `FreqDist()`. 

In the cell below:

* Create an empty list called `articles_concat` 
* Iterate through `processed_data` and add every article it contains to `articles_concat` 
* Pass `articles_concat` as input to `FreqDist()`  
* Display the top 200 most used words  

In [ ]:
articles_concat = None

In [ ]:
articles_freqdist = None


At first glance, none of these words seem very informative -- for most of the words represented here, it would be tough to guess if a given word is used equally among all five classes, or is disproportionately represented among a single class. This makes sense, because this frequency distribution represents all the classes combined. This tells us that these words are probably the least important, as they are most likely words that are used across multiple classes, thereby providing our model with little actual signal as to what class they belong to. This tells us that we probably want to focus on words that appear heavily in articles from a given class, but rarely appear in articles from other classes. You may recall from previous lessons that this is exactly where **_TF-IDF Vectorization_** really shines!

### Vectorizing with TF-IDF

Although NLTK does provide functionality for vectorizing text documents with TF-IDF, we'll make use of scikit-learn's TF-IDF vectorizer, because we already have experience with it, and because it's a bit easier to use, especially when the models we'll be feeding the vectorized data into are from scikit-learn, meaning that we don't have to worry about doing any extra processing to ensure they play nicely together. 

Recall that in order to use scikit-learn's `TfidfVectorizer()`, we need to pass in the data as raw text documents -- the `TfidfVectorizer()` handles the count vectorization process on it's own, and then fits and transforms the data into TF-IDF format. 

This means that we need to:

* Import `TfidfVectorizer` from `sklearn.feature_extraction.text` and instantiate `TfidfVectorizer()` 
* Call the vectorizer object's `.fit_transform()` method and pass in our `data` as input. Store the results in `tf_idf_data_train` 
* Also create a vectorized version of our testing data, which can be found in `newsgroups_test.data`. Store the results in `tf_idf_data_test`. 


**_NOTE:_** When transforming the test data, use the `.transform()` method, not the `.fit_transform()` method, as the vectorizer has already been fit to the training data. 

In [ ]:
# Import TfidfVectorizer

In [ ]:
vectorizer = None

In [ ]:
tf_idf_data_train = None

In [ ]:
tf_idf_data_test = None

### Modeling Our Data

Great! We've now preprocessed and explored our dataset, let's take a second to see what our data looks like in vectorized form. 

In the cell below, get the shape of `tf_idf_data`.

In [16]:
# Your code here

(2814, 36622)

Our vectorized data contains 2,814 articles, with 36,622 unique words in the vocabulary. However, the vast majority of these columns for any given article will be zero, since every article only contains a small subset of the total vocabulary. Recall that vectors mostly filled with zeros are referred to as **_Sparse Vectors_**. These are extremely common when working with text data. 

Let's check out the average number of non-zero columns in the vectors. Run the cell below to calculate this average. 

In [ ]:
non_zero_cols = tf_idf_data_train.nnz / float(tf_idf_data_train.shape[0])
print("Average Number of Non-Zero Elements in Vectorized Articles: {}".format(non_zero_cols))

percent_sparse = 1 - (non_zero_cols / float(tf_idf_data_train.shape[1]))
print('Percentage of columns containing 0: {}'.format(percent_sparse))

As we can see from the output above, the average vectorized article contains 107 non-zero columns. This means that 99.7% of each vector is actually zeroes! This is one reason why it's best not to create your own vectorizers, and rely on professional packages such as scikit-learn and NLTK instead -- they contain many speed and memory optimizations specifically for dealing with sparse vectors. This way, we aren't wasting a giant chunk of memory on a vectorized dataset that only has valid information in 0.3% of it. 

Now that we've vectorized our dataset, let's create some models and fit them to our vectorized training data. 

In the cell below:

* Instantiate `MultinomialNB()` and `RandomForestClassifier()`. For random forest, set `n_estimators` to `100`. Don't worry about tweaking any of the other parameters  
* Fit each to our vectorized training data 
* Create predictions for our training and test sets
* Calculate the `accuracy_score()` for both the training and test sets (you'll find our training labels stored within the variable `target`, and the test labels stored within `newsgroups_test.target`) 

In [ ]:
nb_classifier = None
rf_classifier = None

In [ ]:

nb_train_preds = None
nb_test_preds = None

In [ ]:

rf_train_preds = None
rf_test_preds = None

In [ ]:
nb_train_score = None
nb_test_score = None
rf_train_score = None
rf_test_score = None

print("Multinomial Naive Bayes")
print("Training Accuracy: {:.4} \t\t Testing Accuracy: {:.4}".format(nb_train_score, nb_test_score))
print("")
print('-'*70)
print("")
print('Random Forest')
print("Training Accuracy: {:.4} \t\t Testing Accuracy: {:.4}".format(rf_train_score, rf_test_score))

### Interpreting Results

**_Question:_** Interpret the results seen above. How well did the models do? How do they compare to random guessing? How would you describe the quality of the model fit?

Write your answer below:

In [ ]:
# Your answer here

# Summary

In this lab, we used our NLP skills to clean, preprocess, explore, and fit models to text data for classification. This wasn't easy -- great job!!